In [3]:
import sys, os
print("Kernel Python:", sys.executable)

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
print("PyTorch:", torch.__version__)

from tqdm import tqdm
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print("Current location:", os.getcwd())

from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig

import torch
print(torch.version.cuda)   # shows the CUDA version PyTorch was built against
print(torch.backends.cudnn.version())  # cuDNN version
print(torch.cuda.is_available())

Kernel Python: /work3/s232958/envs/esm_gpu/bin/python
PyTorch: 2.5.1
Using device: cuda
Current location: /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts
12.1
90100
True


In [4]:
interaction_df = pd.read_csv("../data/PPint_DB/disordered_interfaces_no_cutoff_filtered_nonredundant80_3å_5.csv.gz",index_col=0).reset_index(drop=True)
sample_df = interaction_df.sample(n=1500, random_state=0)
all_pdbs = set(sample_df['PDB'])
interactiondf_filtered = interaction_df[interaction_df['PDB'].isin(all_pdbs)].copy()
interactiondf_filtered

,PDB,interface_index,chainname,sequence,interface_residues,disorderscore_predictions,mean disorder interface,seq. len,PDB_interface_name,protien_interface_sequences,interacting_seq_aa,inter_chain_hamming
154,7S4N,0,A,YGTDTCPFPVLANKTNKAKFVGCHQKCNGGDQKLTDGTACYVVERK...,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 15, 16, 17, 79...","[0.3875, 0.4025, 0.3932, 0.3878, 0.3919, 0.376...",0.299444,82,7S4N_0,GRECCLEYFKGAIPLRKLKTWYQTSEDCSRDAIVFVTVQGRAICSD...,GRECCLEYFDCAIC-YGDTCPFPVLNKAKGN,0.195122
155,7S4N,0,B,GRECCLEYFKGAIPLRKLKTWYQTSEDCSRDAIVFVTVQGRAICSD...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 26, 27, 41, 42, 43]","[0.2658, 0.2559, 0.2598, 0.2439, 0.2104, 0.211...",0.179357,63,7S4N_0,GRECCLEYFKGAIPLRKLKTWYQTSEDCSRDAIVFVTVQGRAICSD...,GRECCLEYFDCAIC-YGDTCPFPVLNKAKGN,0.195122
156,5B0O,0,A,TTRLTRWLTALDNFEAKMALLPAVRRYGRLTRATGLVLEATGLQLP...,"[0, 2, 3, 5, 6, 9, 10, 13, 14, 16, 17, 18, 20,...","[0.2376, 0.2192, 0.2178, 0.202, 0.1943, 0.1865...",0.075325,445,5B0O_0,PIHARMQQLVSEFQNTLDALDSVIASRLMQMALEAARQVIGQTPAV...,DAMEGDLDARELLA-TRLRWALFEKMALPAVRRYGRTGR,0.182022
157,5B0O,0,E,PIHARMQQLVSEFQNTLDALDSVIASRLMQMALEAARQVIGQTPAV...,"[20, 24, 28, 112, 113, 114, 115, 116, 117, 122...","[0.2325, 0.2318, 0.2418, 0.2512, 0.2529, 0.235...",0.159864,135,5B0O_0,PIHARMQQLVSEFQNTLDALDSVIASRLMQMALEAARQVIGQTPAV...,DAMEGDLDARELLA-TRLRWALFEKMALPAVRRYGRTGR,0.182022
158,5B0O,1,E,PIHARMQQLVSEFQNTLDALDSVIASRLMQMALEAARQVIGQTPAV...,"[4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 16, 27, 28,...","[0.2325, 0.2318, 0.2418, 0.2512, 0.2529, 0.235...",0.139750,135,5B0O_1,PIHARMQQLVSEFQNTLDALDSVIASRLMQMALEAARQVIGQTPAV...,IMLLMMALAAVIGQVLQILLGGCKV-RMQQLVEFQTLLMMALAARV...,0.985185
...,...,...,...,...,...,...,...,...,...,...,...,...
49431,6WDS,0,H,VQLVESGGGLVKPGGLRLSCAASGFTFSTYIMTWVRQAPGRGLEWV...,"[41, 42, 43, 44, 56, 57, 58, 92, 100, 101, 102...","[0.7256, 0.7606, 0.7419, 0.7242, 0.7088, 0.671...",0.074329,115,6WDS_0,SVLTQPPSASGTPGQRVTISCSGSSSNIEYNYVYWYQKFPGTAPKL...,YTAPKLYAAWDISGVVFG-GLEWFYAYRAYNLWGQG,0.217391
49432,6WDS,2,B,SDRVLQLKLGNSAIVTQEAANYCCAYGEWPNYLPDHEAVAIDKPTQ...,"[20, 21, 22, 23, 24, 25, 26, 27, 36, 63, 64, 6...","[0.2451, 0.2123, 0.2245, 0.2213, 0.2161, 0.218...",0.106527,238,6WDS_2,GVPTYLLPGSGQFLTTDDHSSAPALPCFNPTPEMHIPGQVRNMLEV...,MIPGVESMMGMLKNTLCGSFMLSSESSSH-NYCCAYGEETGSTKFH...,0.155102
49433,6WDS,2,C,GVPTYLLPGSGQFLTTDDHSSAPALPCFNPTPEMHIPGQVRNMLEV...,"[33, 35, 36, 37, 48, 49, 50, 51, 52, 62, 63, 6...","[0.6552, 0.6873, 0.7065, 0.7254, 0.7075, 0.717...",0.103852,245,6WDS_2,GVPTYLLPGSGQFLTTDDHSSAPALPCFNPTPEMHIPGQVRNMLEV...,MIPGVESMMGMLKNTLCGSFMLSSESSSH-NYCCAYGEETGSTKFH...,0.155102
49434,1FX5,0,A,SDDLSFKFKNFSQNGKDLSFQGNASVIETGVLQLNKVGNNLPDETG...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 15, 16, 51, 52, 53...","[0.24, 0.2383, 0.2465, 0.2608, 0.2698, 0.272, ...",0.172220,240,1FX5_0,SDDLSFKFKNFSQNGKDLSFQGNASVIETGVLQLNKVGNNLPDETG...,SDDLSFKFKKDIAPW-SDDLSFKFKKDIAPW,0.995833


In [5]:
interaction_df

,PDB,interface_index,chainname,sequence,interface_residues,disorderscore_predictions,mean disorder interface,seq. len,PDB_interface_name,protien_interface_sequences,interacting_seq_aa,inter_chain_hamming
0,6NZA,0,A,MNTVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIH...,"[184, 185, 186, 187, 188, 191, 192, 194, 198, ...","[0.7864, 0.7974, 0.7796, 0.7675, 0.7414, 0.720...",0.038646,461,6NZA_0,MNTVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIH...,GRTHLATLQESGWAMHYAVTGLNFEDVQGAGASMKNDMVNGASGNF...,0.995662
1,6NZA,0,B,TVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIHAL...,"[182, 183, 184, 185, 186, 190, 192, 196, 197, ...","[0.6701, 0.7181, 0.6893, 0.671, 0.6561, 0.6428...",0.037456,459,6NZA_0,MNTVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIH...,GRTHLATLQESGWAMHYAVTGLNFEDVQGAGASMKNDMVNGASGNF...,0.995662
2,9JKA,1,B,VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,"[34, 35, 38, 39, 42, 45, 46, 49, 53, 56, 57, 6...","[0.3113, 0.3238, 0.2933, 0.2873, 0.2903, 0.289...",0.071228,362,9JKA_1,VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,TAGLSFKIGALKASLPVEHHERKARVASMHDMTKAFLEAKQWTLVS...,1.000000
3,9JKA,1,C,VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,"[83, 94, 105, 106, 108, 109, 112, 113, 116, 11...","[0.3113, 0.3238, 0.2933, 0.2873, 0.2903, 0.289...",0.097310,362,9JKA_1,VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,TAGLSFKIGALKASLPVEHHERKARVASMHDMTKAFLEAKQWTLVS...,1.000000
4,8DQ6,1,B,PTLNLFTNIPVDAVTCSDILKDATKAVAKIIGKPESYVMILLNSGV...,"[15, 19, 20, 23, 34, 35, 36, 37, 38, 39, 40, 4...","[0.1487, 0.1646, 0.1496, 0.1519, 0.1521, 0.155...",0.037771,109,8DQ6_1,PTLNLFTNIPVDAVTCSDILKDATKAVAKIIGKPESYVMILLNSGV...,CLKTESYVMILLNFFGY-VPIAFAGGSESSSRFY,0.889908
...,...,...,...,...,...,...,...,...,...,...,...,...
49445,7OXG,0,C,TRYWNAKALPFAFG,"[0, 1, 4, 5, 8, 9, 10, 11, 12]","[0.7407, 0.7988, 0.784, 0.794, 0.8071, 0.832, ...",0.825889,14,7OXG_0,MKVGQDKVVTIRYTLQVEGEVLDQGELSYLHGHRNLIPGLEEALEG...,LDRNLIAYG-TRNALPFAF,0.111111
49446,6UUM,0,F,EIVLTQSPVTLSLSSGETGTLSCRASQNISSSWIAWYQQRRGQVPR...,"[43, 44, 45, 46, 94, 95, 96, 97, 98, 99, 100, ...","[0.3132, 0.2896, 0.2667, 0.2557, 0.2512, 0.253...",0.143800,215,6UUM_0,EIVLTQSPVTLSLSSGETGTLSCRASQNISSSWIAWYQQRRGQVPR...,VPRLSFFTFGPGFIFPSEQSVTSSKSFEC-GLEWKGSDWPGFPLAK...,0.264317
49447,6UUM,0,H,QVQLVQSGAEVRKPGSSVTISCKPVGGTFTNFAIHWVRQAPGQGLE...,"[43, 44, 45, 46, 58, 60, 110, 111, 113, 115, 1...","[0.3256, 0.3418, 0.3422, 0.3367, 0.3317, 0.314...",0.168272,227,6UUM_0,EIVLTQSPVTLSLSSGETGTLSCRASQNISSSWIAWYQQRRGQVPR...,VPRLSFFTFGPGFIFPSEQSVTSSKSFEC-GLEWKGSDWPGFPLAK...,0.264317
49448,8V4Q,1,B,PDFLGHAENPLREEEWARLNETVIQVARRSLVGRRILDIYGPLGAG...,"[39, 40, 42, 43, 54, 145, 152, 164, 185, 186, ...","[0.4081, 0.4024, 0.3923, 0.3909, 0.3986, 0.391...",0.037471,266,8V4Q_1,HAENPLREEEWARLNETVIQVARRSLVGRRILDIYGPLKTIPIIYK...,LHMPDSAAGSHIYEHHHVN-YGLGFGVFHHHTRE,0.883459


In [6]:
interaction_Dict = {}

for _, row in interactiondf_filtered.iterrows():
    key = row['PDB']   
    seq = row['sequence']                    
    interaction_Dict.setdefault(key, []).append(seq)

list(interaction_Dict.items())[:2]

[('7S4N',
  ['YGTDTCPFPVLANKTNKAKFVGCHQKCNGGDQKLTDGTACYVVERKVWDRMTPMLWYECPLGECKNGVCEDLRKKEDCRKGN',
   'GRECCLEYFKGAIPLRKLKTWYQTSEDCSRDAIVFVTVQGRAICSDPNNKRVKNAVKYLQSLE']),
 ('5B0O',
  ['TTRLTRWLTALDNFEAKMALLPAVRRYGRLTRATGLVLEATGLQLPLGATCIIERQDGPETKEVESEVVGFNGQRLFLMPLEEVEGILPGARVYARGKQLPLGPALLGRVLDGGGKPLDGLPAPDTLETGALITPPFNPLQRTPIEHVLDTGVRAINALLTVGRGQRMGLFAGSGVGKSVLLGMMARYTRADVIVVGLIGERGREVKDFIENILGPDGRARSVVIAAPADVSPLLRMQGAAYATRIAEDFRDRGQHVLLIMDSLTRYAMAQREIALAIGEPPATKGYPPSVFAKLPALVERAGNGIHGGGSITAFYTVLTEGDDQQDPIADSARAILDGHIVLSRRLAEAGHYPAIDIEASISRAMTALITEQHYARVRLFKQLLSSFQRNRDLVSVGAYAKGSDPMLDKAITLWPQLEAFLQQGIFERADWEDSLQALDLIFPT',
   'PIHARMQQLVSEFQNTLDALDSVIASRLMQMALEAARQVIGQTPAVDNSALIKQIQQLLQQEPLFSGKPQLRVHPDDLQRVEEMLGATLSLHGWRLRGDPTLHHGGCKVSADEGDLDASVATRWQELCRLAAPGV',
   'PIHARMQQLVSEFQNTLDALDSVIASRLMQMALEAARQVIGQTPAVDNSALIKQIQQLLQQEPLFSGKPQLRVHPDDLQRVEEMLGATLSLHGWRLRGDPTLHHGGCKVSADEGDLDASVATRWQELCRLAAPGV',
   'PIHARMQQLVSEFQNTLDALDSVIASRLMQMALEAARQVIGQTPAVDNSALIKQIQQLLQQEPLFSGKPQLRVHPDDLQ

In [7]:
client = ESMC.from_pretrained("esmc_600m", device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

def calculate_ESM_pr_res_embeddings(sequence):
    protein = ESMProtein(sequence=sequence)
    protein_tensor = client.encode(protein)
    logits_output = client.logits(
    protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
    )
    return logits_output.embeddings.detach().cpu().numpy()

/work3/s232958/envs/esm_gpu/lib/python3.10/site-packages/esm/pretrained.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [8]:
for name, seqs in interaction_Dict.items():
    print(seqs)
    break

['YGTDTCPFPVLANKTNKAKFVGCHQKCNGGDQKLTDGTACYVVERKVWDRMTPMLWYECPLGECKNGVCEDLRKKEDCRKGN', 'GRECCLEYFKGAIPLRKLKTWYQTSEDCSRDAIVFVTVQGRAICSDPNNKRVKNAVKYLQSLE']


In [9]:
# where to save
path_to_output_embeddings = "/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings"
os.makedirs(path_to_output_embeddings, exist_ok=True)

# helper: convert torch tensor to numpy
def to_numpy(x):
    try:
        return x.detach().cpu().numpy()
    except AttributeError:
        return np.asarray(x)

seq_num = ["A", "B", "C", "D", "E", "F", "G", "H"]
for name, seqs in tqdm(interaction_Dict.items(), total=len(interaction_Dict.items()), desc="Embedding targets"):
     for idx, seq in enumerate(seqs[:8]):
        emb = calculate_ESM_pr_res_embeddings(seq)
        emb_np = to_numpy(emb)
        out_path = os.path.join(path_to_output_embeddings, f"{name}_{seq_num[idx]}.npy")
        np.save(out_path, emb_np)
        print(f"Protein {name} embedded and saved to {out_path}")

# Count files in the 
# path_to_output_embeddings = "/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/meta_analysis/targets_embeddings"
if len(os.listdir(path_to_output_embeddings)) == len(interaction_Dict.keys()):
    print("All sequences encoded!")

Embedding targets:   0%|         | 1/1465 [00:03<1:23:08,  3.41s/it]

Protein 7S4N embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7S4N_A.npy
Protein 7S4N embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7S4N_B.npy
Protein 5B0O embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/5B0O_A.npy
Protein 5B0O embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/5B0O_B.npy
Protein 5B0O embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/5B0O_C.npy


Embedding targets:   0%|           | 2/1465 [00:03<39:36,  1.62s/it]

Protein 5B0O embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/5B0O_D.npy
Protein 7OOP embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7OOP_A.npy
Protein 7OOP embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7OOP_B.npy
Protein 7OOP embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7OOP_C.npy
Protein 7OOP embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7OOP_D.npy
Protein 7OOP embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7OOP_E.npy


Embedding targets:   0%|           | 4/1465 [00:04<20:54,  1.16it/s]

...

Protein 7MHY embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7MHY_D.npy
Protein 6WDS embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/6WDS_A.npy
Protein 6WDS embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/6WDS_B.npy


Embedding targets: 100%|███████▉| 1464/1465 [05:32<00:00,  2.79it/s]

Protein 6WDS embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/6WDS_C.npy
Protein 6WDS embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/6WDS_D.npy
Protein 1FX5 embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/1FX5_A.npy


Embedding targets: 100%|████████| 1465/1465 [05:32<00:00,  4.41it/s]

Protein 1FX5 embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/1FX5_B.npy
